# 1. Backfill features

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# functions for replacing date and time with timestamp (seconds since 1970-01-01)

def energy_timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%d.%m.%Y %H:%M')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

def weather_timestamp_2_time(x, i):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = dt_obj + timedelta(hours=i)
    dt_obj = dt_obj.timestamp() * 1000

    return int(dt_obj)

### Load historical data

In [ ]:
## Electricity data

df_energy = pd.read_csv('data/timvarden-2022-01-11-SE3.csv')
df_energy.head()

In [ ]:
# Remove columns with nan/undefined values (Svenska elkraft)

# print(f"rows: {len(df_energy.index)}")

# df_energy_clean = df_energy.drop(['Förbrukning Mwh exkl. avk.last','Förbrukning Mwh >50 MW', 
#                             'Avkopplingsb. last', 'Timmätta förluster', 'Schablonleverans förbrukning', 
#                             'Schablonleverans förluster'], 
#                         axis=1)
# df_energy_clean.rename(columns={"Datum": "tid", "Day-ahead Price [EUR/MWh]": "day_ahead_price"}, inplace=True)
# df_energy_clean.head()

# Remove columns with nan/undefined values (Entsoe)

print(f"rows: {len(df_energy.index)}")
print("Amount n/e in columns")
for col in df_energy.columns:
    print(f"{col}: {df_energy[df_energy[col] == 'n/e'].shape[0]}")

# df_energy_clean = df_energy.drop(['Biomass','Fossil Brown coal/Lignite', 
#                             'Fossil Coal-derived gas', 'Other renewable',
#                             'Fossil Hard coal', 'Fossil Oil', 'Waste',
#                             'Fossil Oil shale', 'Fossil Peat', 'Wind Offshore',
#                             'Hydro Run-of-river and poundage', 'Geothermal',
#                             'Hydro Pumped Storage', 'Hydro Pumped Storage - Actual Consumption',
#                             'Currency'], 
#                         axis=1)
df_energy.head()

In [ ]:
# Convert feature 'Datum' into 'Time'
df_energy_final = df_energy_clean.copy(deep=True)

df_energy_final.iloc[:,0] = df_energy_final.iloc[:,0].apply(energy_timestamp_2_time)
df_energy_final.head()

# for idx in range(len(df_energy_final.index)): 
#     date_time = str(df_energy_final.iloc[idx][0])
#     date_time = date_time.split(" - ")[0]
    
#     print(date_time)
#     # df_energy_final.at[idx, 'MTU'] = energy_timestamp_2_time(date_time)

In [ ]:
## Wheather data
df_weather = pd.read_csv('data/smhi-opendata_2022.csv')

df_weather.head()

In [ ]:
# Add additional rows for hours between 6 and 18 (only for stockholm-observatoriet data)
# Combine features 'datum' and 'tid' into one single

df_new_weather = pd.DataFrame(columns=["datum", "Lufttemperatur", "Kvalitet"])

for idx in range(len(df_weather.index)-1):

    # retrieve date and temperature differece between two times
    date_time = str(df_weather.iloc[idx][0]) + " " + str(df_weather.iloc[idx][1])
    temp_diff = (df_weather.iloc[idx+1][2] - df_weather.iloc[idx][2]) / 12

    # add current row to new df, covert to new format
    new_date_time = weather_timestamp_2_time(date_time, 0)
    df_new_weather.loc[len(df_new_weather)] = [new_date_time, df_weather.loc[idx][2], df_weather.loc[idx][3]]
    #print(date_time)

    for i in range(1, 12):

        # increment time
        new_date_time = weather_timestamp_2_time(date_time, i)
        # dt_obj = datetime.strptime(str(date_time), '%Y-%m-%d %H:%M:%S')
        # dt_obj = dt_obj + timedelta(hours=i)
        # print(idx + i)

        # create additional row
        # time = dt_obj.time()
        # date = dt_obj.date()
        temp = df_weather.iloc[idx][2] + (temp_diff * i)
        qlty = "G"

        # add row to new df
        # print(idx + i)
        row = [new_date_time, temp, qlty]
        df_new_weather.loc[len(df_new_weather)] = row

        #df_weather_copy = df_weather_copy.append(row, ignore_index=True) # date, time, temp, qlty

df_new_weather.head(5)

In [ ]:
df_energy_final.info()
df_new_weather.info()

### Connect to hopsworks

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

### Creating Feature Groups


In [ ]:
electricity_data_fg = fs.get_or_create_feature_group(
        name = 'electricity_data_fg',
        description = 'Energy generation, consumption and price of each hour',
        version = 1,
        primary_key = ['tid'],
        online_enabled = True,
        event_time = 'tid'
    )    

electricity_data_fg.insert(df_energy_final)

In [ ]:
weather_fg = fs.get_or_create_feature_group(
        name = 'weather_fg',
        description = 'Weather characteristics of each day',
        version = 1,
        primary_key = ['city','date'],
        online_enabled = True,
        event_time = 'date'
    )    

weather_fg.insert(df_weather)

### old code

In [ ]:
for idx in df_weather.index:
    date_time = str(df_weather.iloc[idx][0]) + " " + str(df_weather.iloc[idx][1])
    for i in range(1,12):
        dt_obj = datetime.strptime(str(date_time), '%Y-%m-%d %H:%M:%S')
        dt_obj = dt_obj + timedelta(hours=i)

        # append to dataframe
        time = dt_obj.time
        date = dt_obj.date
        print(time, date)
